Based on the full data subset A with no missing data, the full samples for y=0 and y=1 are learned and new samples are generated using the data and labels from tp2.

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from torch.utils.data import TensorDataset, DataLoader
from skbio.stats.composition import clr, alr, ilr
from scipy.cluster.hierarchy import linkage, fcluster, cut_tree
from scipy.spatial.distance import pdist, cdist
from scipy.stats.mstats import gmean
from scipy.stats import gaussian_kde, gamma, dirichlet, multivariate_normal
from scipy.special import softmax
import scipy.stats as stats
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
def evaluate_model(y_true, y_pred, y_pred_proba):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = recall_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    balanced_accuracy = (sensitivity + specificity) / 2
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    
    return {
#         'Accuracy': accuracy,
        'Balanced_accuracy': balanced_accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
#         'F1 Score': f1,
        'AUC': auc
    }

In [3]:
pd.set_option('future.no_silent_downcasting', True)

In [4]:
os.environ["PYTHONHASHSEED"] = "0"

In [5]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.base")

In [6]:
# Define K-fold cross-validation
# 1. There are positive samples in both the training and validation sets.
# 2. the class distributions in the training and validation sets are similar to the original dataset.

n_splits = 5
num_seeds = 5
skf = StratifiedKFold(n_splits= n_splits, shuffle=True, random_state = 42)

### metadata

In [7]:
Metadata = pd.read_csv("data/BASIC_metadata_full.csv", sep=',', low_memory=False)
Metadata = Metadata.drop(Metadata.columns[0], axis=1)

# convert timepoits to 0,1,2
Metadata.loc[Metadata.TimePoint == "Trimester2","TimePoint"] = 0 
Metadata.loc[Metadata.TimePoint == "Trimester3","TimePoint"] = 1
Metadata.loc[Metadata.TimePoint == "PostpartumWeek6","TimePoint"] = 2

# turn insufficient reads to NaN
i = Metadata[Metadata.ReadsNumber < 500000].index
Metadata.loc[i, 'ReadsNumber'] = np.nan

### species data

In [8]:
profile =pd.read_csv("data/Species_Profile_full.csv",sep=',',low_memory=False)

# extract all bacteria names
full_list_bacteria = list(profile.columns)[1:]

species = profile.to_numpy()[:,1:]

species_num = np.shape(species)[1] # 713 species

In [9]:
# Inner join profile and metadeata
merged_data_base = pd.merge(profile, Metadata, left_on='Sample_id', right_on='Sample_ID')

merged_data = merged_data_base.dropna(subset=['ReadsNumber'])[['Individual_ID', 'TimePoint', 'EPDS', 'Dichotomous_EPDS'] + full_list_bacteria]

In [10]:
# 1. Sample individuals whose EPDS != NaN at tp2
individuals_with_na_epds_at_tp2 = merged_data[
    (merged_data['TimePoint'] == 2) & (merged_data['EPDS'].isna())
]['Individual_ID'].unique()

data = merged_data[~merged_data['Individual_ID'].isin(individuals_with_na_epds_at_tp2)]

# 2. Sample individuals with data at tp0, tp1 and tp2
individuals_with_all_timepoints = data.groupby('Individual_ID').filter(lambda x: set(x['TimePoint']) >= {0, 1, 2})['Individual_ID'].unique()
data = data[data['Individual_ID'].isin(individuals_with_all_timepoints)]

In [11]:
# Remove features that have a value of 0 at all time points for all samples
columns_to_drop = []
for col in full_list_bacteria:
    if (data[col] == 0).all():
        columns_to_drop.append(col)
data = data.drop(columns=columns_to_drop)

# Update full_list_bacteria
full_list_bacteria = [col for col in full_list_bacteria if col not in columns_to_drop]

### （1）Pre-Processing

In [12]:
# 1.Replace the zero value by 1/2 of the non - zero minimum value
data[full_list_bacteria] = data[full_list_bacteria].astype(float)

# Replace with 1/2 of the non-zero minimum value of the row
matrix = data[full_list_bacteria].values
for i in range(matrix.shape[0]):
    row = matrix[i, :]
    non_zero_values = row[~np.isnan(row) & (row > 0)]
    if len(non_zero_values) > 0:
        min_non_zero = np.min(non_zero_values)
        half_min = min_non_zero / 2
        row[row == 0] = half_min
    matrix[i, :] = row

data[full_list_bacteria] = matrix

In [13]:
# Normalization
for i in range(matrix.shape[0]):
    row = matrix[i, :]
    
    row_sum = np.sum(row)
    row = row / row_sum
    matrix[i, :] = row

data[full_list_bacteria] = matrix

In [14]:
# Turn data from [individual * tp, feature] into [individual, tp, feature]
grouped = data.groupby('Individual_ID')

transformed_data = []
labels = []

for individual_id, group in grouped:
    time_point_matrix = np.full((3, len(full_list_bacteria)), np.nan)

    for _, row in group.iterrows():
        time_point = int(row['TimePoint'])
        time_point_matrix[time_point] = row[full_list_bacteria].values

    tp2_row = group[group['TimePoint'] == 2]
    label = tp2_row['Dichotomous_EPDS'].values[0]

    transformed_data.append(time_point_matrix)
    labels.append(label)

transformed_data = np.array(transformed_data)
labels = np.array(labels)

In [15]:
print("minority class：", len(labels[labels == 1]))
print("majority class：", len(labels[labels == 0]))

minority class： 15
majority class： 70


In [16]:
# Build an RF model on the data to make predictions with only CLR.
# There are a lot of NAs in features
X = transformed_data[:, :2, :].reshape(transformed_data.shape[0], -1)
y = labels

metrics_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        metrics = evaluate_model(y_val, y_pred, y_pred_proba)
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Average Results:
  Balanced_accuracy: 0.5000 ± 0.0000
  Sensitivity: 0.0000 ± 0.0000
  Specificity: 1.0000 ± 0.0000
  AUC: 0.5871 ± 0.1759


### （2）Feature Extraction + OverSampling

In [17]:
def extract_features(train_data, k):
    selected_features_tp0 = []
    selected_features_tp1 = []
    
    for time_point in [0, 1]:
        time_point_data = train_data[train_data['TimePoint'] == time_point]
        
        X = time_point_data.drop(['Individual_ID', 'TimePoint', 'EPDS', 'Dichotomous_EPDS'], axis=1)
        y = time_point_data['Dichotomous_EPDS']

        selector = SelectKBest(score_func=f_classif, k=k) 
        X_new = selector.fit_transform(X, y)

        feature_indices = selector.get_support(indices=True)
        feature_columns = X.columns[feature_indices]
        
        if time_point == 0:
            selected_features_tp0 = feature_columns
        elif time_point == 1:
            selected_features_tp1 = feature_columns
            
    feature_columns = list(set(selected_features_tp0) | set(selected_features_tp1))
    print(f"Selected features: {feature_columns}")
    
    return feature_columns

In [18]:
def extracted_datasets(data, feature_columns):
    microbiome_columns = data.columns[4:]
    
    unused_bacteria_columns = [col for col in microbiome_columns if col not in feature_columns]
    
    others_column = data[unused_bacteria_columns].sum(axis=1).to_frame(name='Others')
    
    extracted_data = data[['Individual_ID', 'TimePoint', 'EPDS', 'Dichotomous_EPDS'] + feature_columns]
    
    extracted_data = pd.concat([extracted_data, others_column], axis=1)
    
    return extracted_data

In [19]:
def clr_transform(data):
    data_copy = data.copy()
    id_columns = data_copy.columns[:4]
    microbiome_columns = data_copy.columns[4:]

    for _, group in data_copy.groupby('TimePoint'):
        indices = group.index
        group_data = group[microbiome_columns].values
        clr_values = clr(group_data)
        data_copy.loc[indices, microbiome_columns] = clr_values

    return data_copy

In [20]:
def reshape_and_extract_labels(data):
    feature_columns = data.columns[4:]
    
    grouped = data.groupby('Individual_ID')

    extracted_data_final = []
    labels = []

    for individual_id, group in grouped:
        time_point_matrix = np.full((3, len(feature_columns)), np.nan)

        for _, row in group.iterrows():
            time_point = int(row['TimePoint'])
            time_point_matrix[time_point] = row[feature_columns].values

        tp2_row = group[group['TimePoint'] == 2]
        if not tp2_row.empty:  
            label = tp2_row['Dichotomous_EPDS'].values[0]
            labels.append(label)
            extracted_data_final.append(time_point_matrix)

    extracted_data_final = np.array(extracted_data_final)
    labels = np.array(labels)

    return extracted_data_final, labels

In [21]:
# Assuming the number of new samples generated
num_new_samples = 55 # 70-15 = 55
original_num_time_steps = 2
minority_class = 1

In [22]:
individual_ids = np.unique(data['Individual_ID'])
y_for_fold = labels

### cGAN

In [23]:
class Generator(nn.Module):
    def __init__(self, latent_dim, output_shape, cond_dim, tp2_dim):
        super(Generator, self).__init__()
        self.output_shape = output_shape
        self.cond_dim = cond_dim
        self.tp2_dim = tp2_dim

        # Calculate input feature dimensions
        input_dim = latent_dim + cond_dim + tp2_dim

        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),  # Input layer
            nn.LeakyReLU(0.2),
            nn.Linear(64, 128),  # Hidden layer
            nn.LeakyReLU(0.2),
            nn.Linear(128, np.prod(output_shape)),  # Output layer
            nn.Tanh()  # Output range [-1, 1]
        )

    def forward(self, z, c, tp2):
        zctp2 = torch.cat([z, c, tp2], dim=1)
        output = self.model(zctp2)
        output = output.view(output.size(0), *self.output_shape) 
        return output


class Discriminator(nn.Module):
    def __init__(self, input_shape, cond_dim, tp2_dim):
        super(Discriminator, self).__init__()
        self.input_shape = input_shape
        self.cond_dim = cond_dim
        self.tp2_dim = tp2_dim

        # Calculate input feature dimensions
        input_dim = np.prod(input_shape) + cond_dim + tp2_dim

        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),  # Input layer
            nn.LeakyReLU(0.2),
            nn.Linear(128, 64),  # Hidden layer
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),  # Output layer
            nn.Sigmoid()  # Output range [0, 1]
        )

    def forward(self, x, c, tp2):
        x = x.view(x.size(0), -1)
        xctp2 = torch.cat([x, c, tp2], dim=1)
        return self.model(xctp2)
    
class cGAN:
    def __init__(self, latent_dim, output_shape, cond_dim, tp2_dim, lr=0.0002, b1=0.5, b2=0.999):
        self.latent_dim = latent_dim
        self.output_shape = output_shape
        self.cond_dim = cond_dim
        self.tp2_dim = tp2_dim

        # Initialize generator and discriminator
        self.generator = Generator(latent_dim, output_shape, cond_dim, tp2_dim)
        self.discriminator = Discriminator(output_shape, cond_dim, tp2_dim)

        # Define optimizers
        self.optimizer_G = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        self.optimizer_D = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))

        # Define loss function
        self.loss_fn = nn.BCELoss()

    def train(self, X_train, y_train, tp2_train, epochs, batch_size):
        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.float32)
        tp2_train = torch.tensor(tp2_train, dtype=torch.float32)

        y_train_onehot = torch.nn.functional.one_hot(y_train.long(), num_classes=self.cond_dim).float()

        dataset = TensorDataset(X_train, y_train_onehot, tp2_train)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            for real_data, real_c, real_tp2 in dataloader:
                # Labels for real samples
                real_labels = torch.ones(real_data.size(0), 1)
                # Generate fake samples
                noise = torch.randn(real_data.size(0), self.latent_dim)
                fake_data = self.generator(noise, real_c, real_tp2)
                # Labels for fake samples
                fake_labels = torch.zeros(real_data.size(0), 1)

                # Train discriminator
                self.optimizer_D.zero_grad()
                real_loss = self.loss_fn(self.discriminator(real_data, real_c, real_tp2), real_labels)
                fake_loss = self.loss_fn(self.discriminator(fake_data.detach(), real_c, real_tp2), fake_labels)
                d_loss = (real_loss + fake_loss) / 2
                d_loss.backward()
                self.optimizer_D.step()

                # Train generator
                self.optimizer_G.zero_grad()
                g_loss = self.loss_fn(self.discriminator(fake_data, real_c, real_tp2), real_labels)
                g_loss.backward()
                self.optimizer_G.step()

    def generate_samples(self, num_samples, minority_class, tp2_samples):
        self.generator.eval()
        # Generate class labels
        new_c = torch.ones(num_samples, dtype=torch.long) * minority_class
        new_c = torch.nn.functional.one_hot(new_c, num_classes=self.cond_dim).float()
        # Generate random noise
        noise = torch.randn(num_samples, self.latent_dim)
        # Convert tp2_samples to torch.Tensor
        tp2_samples = torch.tensor(tp2_samples, dtype=torch.float32)
        # Generate new samples
        with torch.no_grad():
            fake_data = self.generator(noise, new_c, tp2_samples)
        return fake_data.numpy()

In [24]:
metrics_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(individual_ids, y_for_fold)):
    train_ids = individual_ids[train_idx]
    val_ids = individual_ids[val_idx]

    train_data = data[data['Individual_ID'].isin(train_ids)]
    val_data = data[data['Individual_ID'].isin(val_ids)]

    extracted_data_base = train_data.dropna(subset=['Dichotomous_EPDS'])

    feature_columns = extract_features(extracted_data_base, k=4)

    extracted_data = extracted_datasets(data, feature_columns)
    extracted_data = clr_transform(extracted_data)

    extracted_data_final, labels = reshape_and_extract_labels(extracted_data)

    X = extracted_data_final[:, :2, :].reshape(extracted_data_final.shape[0], -1)
    y = labels
    
    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train = y[train_idx]
    y_val = y[val_idx]
    
    original_num_features = extracted_data_final.shape[2]
    
    minority_class = 1
    minority_indices = np.where(y_train == minority_class)[0]
    tp2_data = extracted_data_final[:, 2, :].reshape(len(extracted_data_final), -1) 
    tp2_train = tp2_data[train_idx]
    tp2_minority = tp2_train[minority_indices]
    
    repeat_times = num_new_samples // len(tp2_minority) + 1
    tp2_minority = np.tile(tp2_minority, (repeat_times, 1))[:num_new_samples]

    latent_dim = 100
    output_shape = (original_num_time_steps, original_num_features)
    cond_dim = 2  # two classes
    cgan = cGAN(latent_dim, output_shape, cond_dim, tp2_train.shape[1])  

    cgan.train(X_train, y_train, tp2_train, epochs=500, batch_size=16)

    new_samples_np = cgan.generate_samples(num_new_samples, minority_class=1, tp2_samples=tp2_minority)
    new_samples_np = new_samples_np.reshape(num_new_samples, -1)

    combined_X_train = np.concatenate([X_train, new_samples_np], axis=0)
    combined_y_train = np.concatenate([y_train, np.full(num_new_samples, minority_class)], axis=0)

    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(combined_X_train, combined_y_train)
        
        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        metrics = evaluate_model(y_val, y_pred, y_pred_proba)
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Pseudoflavonifractor_sp_An184', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Coprococcus_eutactus', 'Roseburia_sp_CAG_303', 'Streptococcus_vestibularis', 'Enterococcus_faecalis', 'Streptococcus_sp_F0442', 'Butyricimonas_synergistica']
Selected features: ['Sellimonas_intestinalis', 'Clostridium_perfringens', 'Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Bifidobacterium_animalis', 'Butyricimonas_synergistica']
Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Megamonas_funiformis_CAG_377', 'Denitrobacterium_detoxificans', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Enteroco

### cGMMs

In [25]:
class ConditionalGaussianMixtureModel:
    
    def __init__(self, n_components=2, max_iter=100, tol=1e-3, random_state=None): 
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        np.random.seed(random_state)
        
    def _initialize_parameters(self, X, C):
        n_samples, n_features = X.shape
        n_conditions = C.shape[1]
        
        # Initialize using K-means
        kmeans = KMeans(n_clusters=self.n_components, random_state=self.random_state)
        labels = kmeans.fit_predict(X)
        
        # Initialize weight parameters (related to conditional variables)
        self.weight_coeffs = np.random.randn(self.n_components, n_conditions + 1)
        
        # Initialize mean parameters (related to conditional variables)
        self.mean_coeffs = np.zeros((self.n_components, n_features, n_conditions + 1))
        for k in range(self.n_components):
            mask = (labels == k)
            if np.sum(mask) > 0:
                self.mean_coeffs[k, :, 0] = np.mean(X[mask], axis=0)
        
        # Initialize covariance parameters (now dependent on conditional variables)
        # For each component, each unique element of covariance matrix has its own coefficients
        self.n_cov_params = (n_features * (n_features + 1)) // 2  # Number of unique elements in symmetric matrix
        self.cov_coeffs = np.zeros((self.n_components, self.n_cov_params, n_conditions + 1))
        
        # Initialize with empirical covariances
        for k in range(self.n_components):
            mask = (labels == k)
            if np.sum(mask) > 0:
                diff = X[mask] - self.mean_coeffs[k, :, 0]
                emp_cov = np.dot(diff.T, diff) / np.sum(mask)
            else:
                emp_cov = np.eye(n_features)
                
            # Ensure positive definite
            emp_cov += 1e-6 * np.eye(n_features)
            
            # Convert covariance matrix to vector form (Cholesky decomposition for stability)
            try:
                chol = np.linalg.cholesky(emp_cov)
                cov_vec = self._chol_to_vec(chol)
                self.cov_coeffs[k, :, 0] = cov_vec
            except np.linalg.LinAlgError:
                # If Cholesky fails, use identity
                chol = np.eye(n_features)
                cov_vec = self._chol_to_vec(chol)
                self.cov_coeffs[k, :, 0] = cov_vec
                
    def _chol_to_vec(self, chol):
    """
        Convert lower triangular Cholesky factor to vector
    """
        n = chol.shape[0]
        vec = []
        for i in range(n):
            for j in range(i + 1):
                vec.append(chol[i, j])
        return np.array(vec)
    
    def _vec_to_chol(self, vec):
    """
        Convert vector back to lower triangular Cholesky factor
    """
        n = int((np.sqrt(1 + 8 * len(vec)) - 1) / 2)
        chol = np.zeros((n, n))
        idx = 0
        for i in range(n):
            for j in range(i + 1):
                chol[i, j] = vec[idx]
                idx += 1
        return chol
    
    def _compute_covariances(self, C_bias):
    """
        Calculate covariance matrices for each sample under each component
    """
        n_samples = C_bias.shape[0]
        n_features = self.mean_coeffs.shape[1]
        
        covariances = np.zeros((n_samples, self.n_components, n_features, n_features))
        
        for k in range(self.n_components):
            # Compute covariance coefficients for each sample
            cov_vecs = np.dot(C_bias, self.cov_coeffs[k].T)  # (n_samples, n_cov_params)
            
            for i in range(n_samples):
                # Convert vector back to Cholesky factor
                chol = self._vec_to_chol(cov_vecs[i])
                
                # Ensure diagonal elements are positive for numerical stability
                for d in range(n_features):
                    if chol[d, d] <= 0:
                        chol[d, d] = 1e-6
                
                # Compute covariance matrix as Chol * Chol^T
                covariances[i, k] = np.dot(chol, chol.T)
                
                # Add regularization for numerical stability
                covariances[i, k] += 1e-8 * np.eye(n_features)
                
        return covariances
    
    def _e_step(self, X, C):
    """
        E-step: Calculate posterior probability of each sample belonging to each component
    """
        n_samples = X.shape[0]
        log_prob = np.zeros((n_samples, self.n_components))
        
        # Calculate weight for each component (dependent on conditional variables)
        C_bias = np.hstack([np.ones((n_samples, 1)), C])
        weights = self._compute_weights(C_bias)
        
        # Calculate mean for each component (dependent on conditional variables)
        means = self._compute_means(C_bias)
        
        # Calculate covariance for each component (dependent on conditional variables)
        covariances = self._compute_covariances(C_bias)
        
        # Calculate log probability of each sample under each component
        for k in range(self.n_components):
            for i in range(n_samples):
                try:
                    log_prob[i, k] = np.log(weights[i, k] + 1e-10) + multivariate_normal.logpdf(
                        X[i], mean=means[i, k], cov=covariances[i, k], allow_singular=True)
                except:
                    # If computation fails, use a default small probability
                    log_prob[i, k] = np.log(weights[i, k] + 1e-10) - 1e6
        
        # Calculate posterior probabilities (responsibilities)
        log_prob_max = np.max(log_prob, axis=1, keepdims=True)
        log_prob -= log_prob_max
        prob = np.exp(log_prob)
        responsibilities = prob / (np.sum(prob, axis=1, keepdims=True) + 1e-10)
        
        return responsibilities
    
    def _m_step(self, X, C, responsibilities):
    """
        M-step: Update model parameters
    """
        n_samples, n_features = X.shape
        n_conditions = C.shape[1]
        
        # Add bias term for M-step
        C_bias = np.hstack([np.ones((n_samples, 1)), C])
        
        # Update weight coefficients
        for k in range(self.n_components):
            # Set up weighted least squares problem
            W = np.sqrt(responsibilities[:, k] + 1e-10)
            y = W
            X_weighted = W[:, np.newaxis] * C_bias
            
            # Solve weighted least squares problem
            try:
                self.weight_coeffs[k] = np.linalg.lstsq(X_weighted, y, rcond=None)[0]
            except:
                # If lstsq fails, keep previous values
                pass
        
        # Update mean coefficients
        for k in range(self.n_components):
            for d in range(n_features):
                # Set up weighted least squares problem
                W = np.sqrt(responsibilities[:, k] + 1e-10)
                y = W * X[:, d]
                X_weighted = W[:, np.newaxis] * C_bias
                
                # Solve weighted least squares problem
                try:
                    self.mean_coeffs[k, d] = np.linalg.lstsq(X_weighted, y, rcond=None)[0]
                except:
                    # If lstsq fails, keep previous values
                    pass
        
        # Update covariance coefficients (now dependent on conditional variables)
        means = self._compute_means(C_bias)
        
        for k in range(self.n_components):
            # For each covariance parameter, set up weighted least squares
            for p in range(self.n_cov_params):
                # First, we need to compute the target values for this parameter
                target_values = np.zeros(n_samples)
                
                for i in range(n_samples):
                    # Compute residual for this sample
                    residual = X[i] - means[i, k]
                    
                    # Compute outer product
                    outer_prod = np.outer(residual, residual)
                    
                    # Convert to Cholesky form and extract the p-th parameter
                    try:
                        chol_target = np.linalg.cholesky(outer_prod + 1e-6 * np.eye(n_features))
                        target_vec = self._chol_to_vec(chol_target)
                        target_values[i] = target_vec[p]
                    except:
                        # If Cholesky fails, use identity contribution
                        chol_target = np.eye(n_features)
                        target_vec = self._chol_to_vec(chol_target)
                        target_values[i] = target_vec[p]
                
                # Set up weighted least squares problem
                W = np.sqrt(responsibilities[:, k] + 1e-10)
                y = W * target_values
                X_weighted = W[:, np.newaxis] * C_bias
                
                # Solve weighted least squares problem
                try:
                    self.cov_coeffs[k, p] = np.linalg.lstsq(X_weighted, y, rcond=None)[0]
                except:
                    # If lstsq fails, keep previous values
                    pass
    
    def _compute_weights(self, C_bias):
    """
        Calculate weight for each sample under each component
    """
        n_samples = C_bias.shape[0]
        logits = np.dot(C_bias, self.weight_coeffs.T)
        
        # Use softmax to ensure weights are positive and sum to 1
        logits_max = np.max(logits, axis=1, keepdims=True)
        logits -= logits_max
        exp_logits = np.exp(logits)
        weights = exp_logits / (np.sum(exp_logits, axis=1, keepdims=True) + 1e-10)
        
        return weights
    
    def _compute_means(self, C_bias):
    """
        Calculate mean for each sample under each component
    """
        n_samples = C_bias.shape[0]
        n_features = self.mean_coeffs.shape[1]
        
        means = np.zeros((n_samples, self.n_components, n_features))
        for k in range(self.n_components):
            for d in range(n_features):
                means[:, k, d] = np.dot(C_bias, self.mean_coeffs[k, d])
                
        return means
    
    def fit(self, X, C):
    """
        Fit conditional Gaussian mixture model using EM algorithm
    """
        # Initialize parameters
        self._initialize_parameters(X, C)
        
        # EM algorithm iterations
        prev_log_likelihood = -np.inf
        
        for iteration in range(self.max_iter):
            # E-step: Calculate posterior probabilities
            responsibilities = self._e_step(X, C)
            
            # M-step: Update parameters
            self._m_step(X, C, responsibilities)
            
            # Calculate log likelihood
            C_bias = np.hstack([np.ones((X.shape[0], 1)), C])
            weights = self._compute_weights(C_bias)
            means = self._compute_means(C_bias)
            covariances = self._compute_covariances(C_bias)
            
            log_likelihood = 0
            for i in range(X.shape[0]):
                component_log_probs = []
                for k in range(self.n_components):
                    try:
                        component_log_prob = np.log(weights[i, k] + 1e-10) + multivariate_normal.logpdf(
                            X[i], mean=means[i, k], cov=covariances[i, k], allow_singular=True)
                    except:
                        component_log_prob = np.log(weights[i, k] + 1e-10) - 1e6
                    component_log_probs.append(component_log_prob)
                
                log_likelihood += self._log_sum_exp(np.array(component_log_probs))
            
            # Check for convergence
            if abs(log_likelihood - prev_log_likelihood) < self.tol:
                break
                
            prev_log_likelihood = log_likelihood
            
        return self
    
    def _log_sum_exp(self, log_probs):
    """
        Calculate log-sum-exp to avoid numerical overflow
    """
        max_log_prob = np.max(log_probs)
        return max_log_prob + np.log(np.sum(np.exp(log_probs - max_log_prob)) + 1e-10)
    
    def predict(self, X, C):
    """
        Predict cluster labels for samples
    """
        responsibilities = self._e_step(X, C)
        return np.argmax(responsibilities, axis=1)
    
    def predict_proba(self, X, C):
    """
        Predict probability of samples belonging to each component
    """
        return self._e_step(X, C)
    
    def sample(self, C, n_samples=1):
    """
        Generate samples based on conditional variables
    """
        rng = np.random.RandomState(self.random_state)
        
        C_bias = np.hstack([np.ones((C.shape[0], 1)), C])
        \n        # Calculate mixture weights
        weights = self._compute_weights(C_bias)
        
        # Calculate mean for each component
        means = self._compute_means(C_bias)
        
        # Calculate covariance for each component
        covariances = self._compute_covariances(C_bias)
        
        # Generate samples
        samples = []
        component_labels = []
        
        samples_per_condition = n_samples // C.shape[0]
        remainder = n_samples % C.shape[0]
        
        for i in range(C.shape[0]):
            # Determine number of samples to generate for current condition
            n_current = samples_per_condition + (1 if i < remainder else 0)
            
            # Randomly select components
            k_samples = np.random.choice(
                self.n_components, 
                size=n_current, 
                p=weights[i]
            )
            
            # Generate samples from each selected component
            for k in k_samples:
                try:
                    sample = np.random.multivariate_normal(
                        means[i, k], 
                        covariances[i, k]
                    )
                except:
                    # If sampling fails, use identity covariance
                    sample = np.random.multivariate_normal(
                        means[i, k], 
                        np.eye(means.shape[2])
                    )
                samples.append(sample)
                component_labels.append(k)
                
        return np.array(samples), np.array(component_labels)

def conditional_gmm(X_train, y_train, tp2_train, minority_class=1, num_new_samples=100, n_components=2, random_state=42):
    # Get indices of minority class samples
    minority_indices = np.where(y_train == minority_class)[0]
    
    # Create conditional variables: tp2 data + labels
    C = np.column_stack([tp2_train, y_train.reshape(-1, 1)])
    
    # Create and fit conditional Gaussian mixture model
    cgmm = ConditionalGaussianMixtureModel(n_components=n_components, random_state=random_state)
    cgmm.fit(X_train, C)
    
    # Prepare conditions for generating samples
    # Method 1: Use conditions from all minority class samples
    minority_conditions = C[minority_indices]
    
    # If number of minority samples is less than requested new samples, reuse conditions
    if len(minority_indices) < num_new_samples:
        # Calculate number of repeats needed
        repeat_times = num_new_samples // len(minority_indices) + 1
        minority_conditions = np.tile(minority_conditions, (repeat_times, 1))[:num_new_samples]
    else:
        # Randomly select conditions from minority class samples
        random_indices = np.random.choice(len(minority_indices), num_new_samples, replace=False)
        minority_conditions = minority_conditions[random_indices]
    
    # Generate new samples using conditions
    new_samples, _ = cgmm.sample(minority_conditions, n_samples=num_new_samples)
    
    # Label all generated samples as minority class
    new_labels = np.full(num_new_samples, minority_class)
    
    return new_samples, new_labels

In [26]:
metrics_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(individual_ids, y_for_fold)):
    train_ids = individual_ids[train_idx]
    val_ids = individual_ids[val_idx]
    train_data = data[data['Individual_ID'].isin(train_ids)]
    val_data = data[data['Individual_ID'].isin(val_ids)]
    extracted_data_base = train_data.dropna(subset=['Dichotomous_EPDS'])
    feature_columns = extract_features(extracted_data_base, k=4)
    extracted_data = extracted_datasets(data, feature_columns)
    extracted_data = clr_transform(extracted_data)
    extracted_data_final, labels = reshape_and_extract_labels(extracted_data)
    X = extracted_data_final[:, :2, :].reshape(extracted_data_final.shape[0], -1)
    y = labels
    
    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train = y[train_idx]
    y_val = y[val_idx]
    
    tp2_data = extracted_data_final[:, 2, :].reshape(len(extracted_data_final), -1)
    tp2_train = tp2_data[train_idx]
    
    minority_class = 1
    minority_count = np.sum(y_train == minority_class)
    majority_count = np.sum(y_train != minority_class)
    
    new_samples, new_labels = conditional_gmm(
        X_train, 
        y_train, 
        tp2_train,
        minority_class=minority_class,
        num_new_samples=num_new_samples,
    )
    
    combined_X_train = np.concatenate([X_train, new_samples], axis=0)
    combined_y_train = np.concatenate([y_train, new_labels], axis=0)
    
    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(combined_X_train, combined_y_train)
        
        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        balanced_accuracy = (sensitivity + specificity) / 2
        auc = roc_auc_score(y_val, y_pred_proba)

        metrics = {
            'Balanced_accuracy': balanced_accuracy,
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'AUC': auc,
        }
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Pseudoflavonifractor_sp_An184', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Coprococcus_eutactus', 'Roseburia_sp_CAG_303', 'Streptococcus_vestibularis', 'Enterococcus_faecalis', 'Streptococcus_sp_F0442', 'Butyricimonas_synergistica']
Selected features: ['Sellimonas_intestinalis', 'Clostridium_perfringens', 'Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Bifidobacterium_animalis', 'Butyricimonas_synergistica']
Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Megamonas_funiformis_CAG_377', 'Denitrobacterium_detoxificans', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Enteroco

### CVAEs

In [27]:
class cVAE(nn.Module):
    def __init__(self, input_dim, latent_dim, cond_dim, tp2_dim):
        super(cVAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.tp2_dim = tp2_dim

        self.encoder = nn.Sequential(
            nn.Linear(input_dim + cond_dim + tp2_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(64, latent_dim)  
        self.fc_logvar = nn.Linear(64, latent_dim) 

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + cond_dim + tp2_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Tanh()
        )

    def encode(self, x, c, tp2):
        xctp2 = torch.cat([x, c, tp2], dim=1)
        h = self.encoder(xctp2)
        mu = self.fc_mu(h) 
        logvar = self.fc_logvar(h)  
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, tp2):
        zctp2 = torch.cat([z, c, tp2], dim=1)
        return self.decoder(zctp2)

    def forward(self, x, c, tp2):
        mu, logvar = self.encode(x, c, tp2)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, tp2)
        return recon_x, mu, logvar

In [28]:
class cVAETrainer:
   def __init__(self, input_dim, latent_dim, cond_dim, tp2_dim, lr=0.001):
       self.input_dim = input_dim
       self.latent_dim = latent_dim
       self.cond_dim = cond_dim
       self.tp2_dim = tp2_dim
       # Initialize model
       self.model = cVAE(input_dim, latent_dim, cond_dim, tp2_dim)
       self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
       self.loss_fn = nn.MSELoss()  # Reconstruction loss
   def train(self, X_train, y_train, tp2_train, epochs, batch_size):
       X_train = torch.tensor(X_train, dtype=torch.float32)
       y_train = torch.tensor(y_train, dtype=torch.long)
       tp2_train = torch.tensor(tp2_train, dtype=torch.float32)
       y_train_onehot = torch.nn.functional.one_hot(y_train, num_classes=self.cond_dim).float()
       dataset = TensorDataset(X_train, y_train_onehot, tp2_train)
       dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
       for epoch in range(epochs):
           total_loss = 0.0
           for batch_X, batch_c, batch_tp2 in dataloader:
               # Forward propagation
               recon_X, mu, logvar = self.model(batch_X, batch_c, batch_tp2)
               # Calculate loss
               recon_loss = self.loss_fn(recon_X, batch_X)  # Reconstruction loss
               kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL divergence
               loss = recon_loss + kl_divergence
               # Backward propagation
               self.optimizer.zero_grad()
               loss.backward()
               self.optimizer.step()
               total_loss += loss.item()
#             print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(dataloader):.4f}")
   def generate_samples(self, num_samples, minority_class, tp2_samples):
       self.model.eval()
       # Generate class labels
       new_c = torch.ones(num_samples, dtype=torch.long) * minority_class
       new_c = torch.nn.functional.one_hot(new_c, num_classes=self.cond_dim).float()
       # Generate random noise
       noise = torch.randn(num_samples, self.latent_dim)
       # Convert tp2_samples to torch.Tensor
       tp2_samples = torch.tensor(tp2_samples, dtype=torch.float32)
       # Generate new samples
       with torch.no_grad():
           fake_data = self.model.decode(noise, new_c, tp2_samples)
       return fake_data.numpy()

In [29]:
metrics_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(individual_ids, y_for_fold)):
    train_ids = individual_ids[train_idx]
    val_ids = individual_ids[val_idx]

    train_data = data[data['Individual_ID'].isin(train_ids)]
    val_data = data[data['Individual_ID'].isin(val_ids)]

    extracted_data_base = train_data.dropna(subset=['Dichotomous_EPDS'])

    feature_columns = extract_features(extracted_data_base, k=4)

    extracted_data = extracted_datasets(data, feature_columns)
    extracted_data = clr_transform(extracted_data)

    extracted_data_final, labels = reshape_and_extract_labels(extracted_data)

    X = extracted_data_final[:, :2, :].reshape(extracted_data_final.shape[0], -1)
    y = labels
    
    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train = y[train_idx]
    y_val = y[val_idx]
    
    original_num_features = extracted_data_final.shape[2]
    
    minority_class = 1
    minority_indices = np.where(y_train == minority_class)[0]
    tp2_data = extracted_data_final[:, 2, :].reshape(len(extracted_data_final), -1)  
    tp2_train = tp2_data[train_idx]
    tp2_minority = tp2_train[minority_indices]
    
    repeat_times = num_new_samples // len(tp2_minority) + 1
    tp2_minority = np.tile(tp2_minority, (repeat_times, 1))[:num_new_samples]

    latent_dim = 10
    input_dim = X_train.shape[1]
    cond_dim = 2  # two classes
    cvae = cVAETrainer(input_dim, latent_dim, cond_dim, tp2_train.shape[1])  

    cvae.train(X_train, y_train, tp2_train, epochs=100, batch_size=16)

    new_samples_np = cvae.generate_samples(num_new_samples, minority_class=1, tp2_samples=tp2_minority)
    new_samples_np = new_samples_np.reshape(num_new_samples, -1)

    combined_X_train = np.concatenate([X_train, new_samples_np], axis=0)
    combined_y_train = np.concatenate([y_train, np.full(num_new_samples, minority_class)], axis=0)

    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(combined_X_train, combined_y_train)
        
        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        metrics = evaluate_model(y_val, y_pred, y_pred_proba)
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Pseudoflavonifractor_sp_An184', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Coprococcus_eutactus', 'Roseburia_sp_CAG_303', 'Streptococcus_vestibularis', 'Enterococcus_faecalis', 'Streptococcus_sp_F0442', 'Butyricimonas_synergistica']
Selected features: ['Sellimonas_intestinalis', 'Clostridium_perfringens', 'Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Bifidobacterium_animalis', 'Butyricimonas_synergistica']
Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Megamonas_funiformis_CAG_377', 'Denitrobacterium_detoxificans', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Enteroco

### Kernel density estimation

https://stackoverflow.com/questions/60223573/conditional-sampling-from-multivariate-kernel-density-estimate-in-python

In [30]:
class MultivariateConditionalKDE:
    """
    Multivariate Conditional KDE for generating new X samples conditioned on tp2 and class label.
    This implementation learns from all classes but generates only minority class samples.
    """
    
    def __init__(self, bandwidth_factor=1.0):
        
        self.bandwidth_factor = bandwidth_factor
        self.X = None  # Features
        self.tp2 = None  # Conditional variables
        self.y = None  # Labels
        self.X_scaler = StandardScaler()
        self.tp2_scaler = StandardScaler()
        self.class_indices = {}  # Dictionary to store indices for each class
        
    def fit(self, X, y, tp2, minority_class=1):
        """
        Fit the conditional KDE model using all samples but remembering class information.
        """
        self.y = np.asarray(y)
        
        # Scale all data (using entire dataset)
        self.X = self.X_scaler.fit_transform(X)
        self.tp2 = self.tp2_scaler.fit_transform(tp2)
        
        # Store indices for each class
        unique_classes = np.unique(y)
        for cls in unique_classes:
            self.class_indices[cls] = np.where(y == cls)[0]
        
        # Store minority class for later use
        self.minority_class = minority_class
        
        # Compute the bandwidth using Silverman's rule
        n_samples = self.X.shape[0]
        self.X_bw = self.bandwidth_factor * np.std(self.X, axis=0) * (n_samples ** (-1/5))
        self.tp2_bw = self.bandwidth_factor * np.std(self.tp2, axis=0) * (n_samples ** (-1/5))
        
        return self
    
    def _compute_weights(self, tp2_condition, target_class):
        """
        Compute weights for each training point of the specified class
        based on similarity to the conditional tp2 value.
        """
        # Get indices for the target class
        class_indices = self.class_indices[target_class]
        
        # Normalize the condition
        tp2_normalized = self.tp2_scaler.transform(tp2_condition.reshape(1, -1))
        
        # Compute distances in tp2 space for the specified class
        class_tp2 = self.tp2[class_indices]
        distances = cdist(tp2_normalized, class_tp2, 'euclidean')
        
        # Apply kernel function to get weights
        weights = np.exp(-0.5 * distances ** 2 / np.mean(self.tp2_bw) ** 2)
        weights = weights.flatten()
        
        # Normalize weights
        if np.sum(weights) > 0:
            weights = weights / np.sum(weights)
        
        return weights, class_indices
    
    def generate_samples(self, tp2_conditions, n_samples_per_condition=1):
        """
        Generate new X samples of the minority class conditional on given tp2 values.
        """   
        all_samples = []
        all_labels = []
        
        for tp2_condition in tp2_conditions:
            # Compute weights for the minority class
            weights, minority_indices = self._compute_weights(tp2_condition, self.minority_class)
            
            # Generate samples by weighted resampling with perturbation
            for _ in range(n_samples_per_condition):
                # Sample a point based on weights
                idx = np.random.choice(len(minority_indices), p=weights)
                original_idx = minority_indices[idx]
                base_point = self.X[original_idx]
                
                # Add Gaussian noise scaled by bandwidth
                noise = np.random.normal(0, self.X_bw)
                new_sample = base_point + noise
                
                all_samples.append(new_sample)
                all_labels.append(self.minority_class)
        
        # Convert back to original scale
        new_X_samples = self.X_scaler.inverse_transform(np.array(all_samples))
        new_y_labels = np.array(all_labels)
        
        return new_X_samples, new_y_labels


def conditional_kde(X_train, y_train, tp2_train, minority_class=1, num_new_samples=100, bandwidth_factor=1.0):
    """
    Generate new samples for the minority class conditioned on tp2 values.
    This function learns from all classes but generates only minority class samples.
    """
    ckde = MultivariateConditionalKDE(bandwidth_factor=bandwidth_factor)
    ckde.fit(X_train, y_train, tp2_train, minority_class=minority_class)
    
    minority_indices = np.where(y_train == minority_class)[0]
    selected_indices = np.random.choice(minority_indices, num_new_samples, replace=True)
    selected_tp2 = tp2_train[selected_indices]
    
    new_samples, new_labels = ckde.generate_samples(
        selected_tp2, 
        n_samples_per_condition=1
    )
    
    return new_samples, new_labels

In [31]:
metrics_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(individual_ids, y_for_fold)):
    train_ids = individual_ids[train_idx]
    val_ids = individual_ids[val_idx]
    train_data = data[data['Individual_ID'].isin(train_ids)]
    val_data = data[data['Individual_ID'].isin(val_ids)]
    extracted_data_base = train_data.dropna(subset=['Dichotomous_EPDS'])
    feature_columns = extract_features(extracted_data_base, k=4)
    extracted_data = extracted_datasets(data, feature_columns)
    extracted_data = clr_transform(extracted_data)
    extracted_data_final, labels = reshape_and_extract_labels(extracted_data)
    X = extracted_data_final[:, :2, :].reshape(extracted_data_final.shape[0], -1)
    y = labels
    
    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train = y[train_idx]
    y_val = y[val_idx]
    
    tp2_data = extracted_data_final[:, 2, :].reshape(len(extracted_data_final), -1)
    tp2_train = tp2_data[train_idx]
    
    minority_class = 1
    minority_count = np.sum(y_train == minority_class)
    majority_count = np.sum(y_train != minority_class)
    
#     print(f"train set: sum={len(y_train)}, minority={minority_count}, majority={majority_count}")
    
    new_samples, new_labels = conditional_kde(
        X_train, 
        y_train, 
        tp2_train,
        minority_class=minority_class,
        num_new_samples=num_new_samples,
    )
    
#     print(f"Generated {len(new_samples)} new samples")
    
    combined_X_train = np.concatenate([X_train, new_samples], axis=0)
    combined_y_train = np.concatenate([y_train, new_labels], axis=0)
    
    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(combined_X_train, combined_y_train)
        
        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        balanced_accuracy = (sensitivity + specificity) / 2
        auc = roc_auc_score(y_val, y_pred_proba)

        metrics = {
            'Balanced_accuracy': balanced_accuracy,
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'AUC': auc,
        }
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Pseudoflavonifractor_sp_An184', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Coprococcus_eutactus', 'Roseburia_sp_CAG_303', 'Streptococcus_vestibularis', 'Enterococcus_faecalis', 'Streptococcus_sp_F0442', 'Butyricimonas_synergistica']
Selected features: ['Sellimonas_intestinalis', 'Clostridium_perfringens', 'Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Bifidobacterium_animalis', 'Butyricimonas_synergistica']
Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Megamonas_funiformis_CAG_377', 'Denitrobacterium_detoxificans', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Enteroco

### Dirichlet Distribution + Bayesian Inference

https://blog.csdn.net/shizheng_Li/article/details/144152742

In [32]:
def conditional_dirichlet(X_train, y_train, tp2_train, minority_class=1, 
                        num_new_samples=55, a_prior=0.5, b_prior=0.5):
   X_scaler = StandardScaler()
   X_scaled = X_scaler.fit_transform(X_train)
   
   tp2_scaler = StandardScaler()
   tp2_scaled = tp2_scaler.fit_transform(tp2_train)
   
   # Ensure all features are positive (requirement for Dirichlet distribution)
   X_min = np.min(X_scaled, axis=0)
   X_positive = np.maximum(X_scaled - X_min + 1e-3, 1e-8)
   
   # Get indices of minority class samples
   minority_indices = np.where(y_train == minority_class)[0]
   tp2_minority = tp2_scaled[minority_indices]
   
   # Randomly select conditional values from minority class
   selected_indices = np.random.choice(len(minority_indices), num_new_samples, replace=True)
   selected_tp2 = tp2_minority[selected_indices]
   
   new_samples = []
   
   # Get class information for analyzing distributions based on y
   unique_classes = np.unique(y_train)
   class_indices = {cls: np.where(y_train == cls)[0] for cls in unique_classes}
   
   # Generate samples for each selected tp2 condition
   for tp2_condition in selected_tp2:
       # Calculate weights based on tp2 condition - using all samples
       tp2_distances = np.sum((tp2_scaled - tp2_condition) ** 2, axis=1) ** 0.5
       tp2_distances = np.maximum(tp2_distances, 1e-8)
       tp2_weights = 1.0 / tp2_distances
       tp2_weights = tp2_weights / np.sum(tp2_weights)
       
       # Calculate alpha parameter for each feature dimension
       alphas = np.zeros(X_positive.shape[1])
       
       for j in range(X_positive.shape[1]):
           # Based on tp2 condition, analyze feature distributions for each class
           class_feature_stats = {}
           for cls in unique_classes:
               cls_indices = class_indices[cls]
               cls_features = X_positive[cls_indices, j]
               cls_weights = tp2_weights[cls_indices]
               
               # Calculate weighted statistics for each class
               weighted_mean = np.sum(cls_weights * cls_features) / np.sum(cls_weights) if np.sum(cls_weights) > 0 else 0
               class_feature_stats[cls] = {
                   'mean': weighted_mean,
                   'indices': cls_indices,
                   'weights': cls_weights
               }
           
           # Calculate feature difference ratio between classes
           if len(unique_classes) > 1 and minority_class in class_feature_stats:
               # Calculate ratio of feature means to represent class differences
               minority_mean = class_feature_stats[minority_class]['mean']
               
               # Calculate mean of all other classes
               other_classes = [c for c in unique_classes if c != minority_class]
               other_means = [class_feature_stats[c]['mean'] for c in other_classes]
               other_mean = np.mean(other_means) if other_means else 0
               
               # Calculate class difference ratio
               class_ratio = minority_mean / (other_mean + 1e-8)
               class_ratio = np.clip(class_ratio, 0.5, 2.0)  # Limit ratio range
           else:
               class_ratio = 1.0
           
           # Calculate weighted posterior parameters using all samples
           weighted_data = np.sum(tp2_weights * X_positive[:, j])
           weighted_log_data = np.sum(tp2_weights * np.log(X_positive[:, j]))
           
           # Bayesian posterior
           a_posterior = a_prior + weighted_data
           b_posterior = b_prior + weighted_log_data
           
           # Use expected value of posterior as alpha
           alpha_j = a_posterior / b_posterior
           
           # Adjust alpha based on class feature differences to make it closer to minority class distribution
           alpha_j *= class_ratio
           
           # Ensure alpha is positive
           alphas[j] = max(alpha_j, 0.01)
       
       sample = np.random.dirichlet(alphas)
       new_samples.append(sample)
   
   new_samples = np.array(new_samples)
   
   new_samples_scaled = new_samples * (np.max(X_scaled, axis=0) - X_min) + X_min
   
   new_samples_orig = X_scaler.inverse_transform(new_samples_scaled)
   
   new_labels = np.full(num_new_samples, minority_class)
   
   return new_samples_orig, new_labels

In [33]:
metrics_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(individual_ids, y_for_fold)):

    train_ids = individual_ids[train_idx]
    val_ids = individual_ids[val_idx]

    train_data = data[data['Individual_ID'].isin(train_ids)]
    val_data = data[data['Individual_ID'].isin(val_ids)]

    extracted_data_base = train_data.dropna(subset=['Dichotomous_EPDS'])
    feature_columns = extract_features(extracted_data_base, k=4)
    extracted_data = extracted_datasets(data, feature_columns)
    extracted_data = clr_transform(extracted_data)
    extracted_data_final, labels = reshape_and_extract_labels(extracted_data)

    X = extracted_data_final[:, :2, :].reshape(extracted_data_final.shape[0], -1)
    y = labels
    tp2_data = extracted_data_final[:, 2, :].reshape(len(extracted_data_final), -1)

    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train = y[train_idx]
    y_val = y[val_idx]
    tp2_train = tp2_data[train_idx]
    
    new_samples, new_labels = conditional_dirichlet(
        X_train, 
        y_train, 
        tp2_train,
        minority_class=1,
        num_new_samples=num_new_samples,
        a_prior=0.5, 
        b_prior=0.8
    )

    combined_X_train = np.concatenate([X_train, new_samples], axis=0)
    combined_y_train = np.concatenate([y_train, new_labels], axis=0)

    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(combined_X_train, combined_y_train)

        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        metrics = evaluate_model(y_val, y_pred, y_pred_proba)
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Pseudoflavonifractor_sp_An184', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Coprococcus_eutactus', 'Roseburia_sp_CAG_303', 'Streptococcus_vestibularis', 'Enterococcus_faecalis', 'Streptococcus_sp_F0442', 'Butyricimonas_synergistica']
Selected features: ['Sellimonas_intestinalis', 'Clostridium_perfringens', 'Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Bifidobacterium_animalis', 'Butyricimonas_synergistica']
Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Megamonas_funiformis_CAG_377', 'Denitrobacterium_detoxificans', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Enteroco

### SMOTE

In [34]:
for fold, (train_idx, val_idx) in enumerate(skf.split(individual_ids, y_for_fold)):
    train_ids = individual_ids[train_idx]
    val_ids = individual_ids[val_idx]

    train_data = data[data['Individual_ID'].isin(train_ids)]
    val_data = data[data['Individual_ID'].isin(val_ids)]

    extracted_data_base = train_data.dropna(subset=['Dichotomous_EPDS'])

    feature_columns = extract_features(extracted_data_base, k=4)

    extracted_data = extracted_datasets(data, feature_columns)
    extracted_data = clr_transform(extracted_data)

    extracted_data_final, labels = reshape_and_extract_labels(extracted_data)

    X_all = extracted_data_final[:, :, :].reshape(extracted_data_final.shape[0], -1)
    X = extracted_data_final[:, :2, :].reshape(extracted_data_final.shape[0], -1)
    y = labels

    X_train_all = X_all[train_idx]
    X_val_all = X_all[val_idx]
    X_train = X[train_idx]
    X_val = X[val_idx]
    y_train = y[train_idx]
    y_val = y[val_idx]

    original_num_features = extracted_data_final.shape[2]

    minority_class = 1
    minority_indices = np.where(y_train == minority_class)[0]
    X_train_minority = X_train[minority_indices]

    smote = SMOTE(random_state=42)
    combined_X_train, combined_y_train = smote.fit_resample(X_train, y_train)

    for seed in range(num_seeds):
        rf = RandomForestClassifier(n_estimators=100, random_state=seed, class_weight="balanced")
        rf.fit(combined_X_train, combined_y_train)

        y_pred = rf.predict(X_val)
        y_pred_proba = rf.predict_proba(X_val)[:, 1]

        metrics = evaluate_model(y_val, y_pred, y_pred_proba)
        metrics_list.append(metrics)

avg_metrics = {
    'Balanced_accuracy': np.mean([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.mean([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.mean([m['Specificity'] for m in metrics_list]),
    'AUC': np.mean([m['AUC'] for m in metrics_list]),
}

std_metrics = {
    'Balanced_accuracy': np.std([m['Balanced_accuracy'] for m in metrics_list]),
    'Sensitivity': np.std([m['Sensitivity'] for m in metrics_list]),
    'Specificity': np.std([m['Specificity'] for m in metrics_list]),
    'AUC': np.std([m['AUC'] for m in metrics_list]),
}

print("Average Results:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.4f} ± {std_metrics[metric]:.4f}")

Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Pseudoflavonifractor_sp_An184', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Coprococcus_eutactus', 'Roseburia_sp_CAG_303', 'Streptococcus_vestibularis', 'Enterococcus_faecalis', 'Streptococcus_sp_F0442', 'Butyricimonas_synergistica']
Selected features: ['Sellimonas_intestinalis', 'Clostridium_perfringens', 'Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Roseburia_sp_CAG_303', 'Enterococcus_faecalis', 'Bifidobacterium_animalis', 'Butyricimonas_synergistica']
Selected features: ['Allisonella_histaminiformans', 'Faecalitalea_cylindroides', 'Enterococcus_faecium', 'Megamonas_funiformis_CAG_377', 'Denitrobacterium_detoxificans', 'Enterococcus_faecalis', 'Butyricimonas_synergistica']
Selected features: ['Lactococcus_lactis', 'Allisonella_histaminiformans', 'Enteroco